추상적 요약을 이용할 때는 레이블링 된 데이터가 있어야함.

추출적 요약으로 얻은 데이터를 라벨로하여 fine-tuning을 시켜보는 건?

In [1]:
# rank.py
import numpy as np
from sklearn.preprocessing import normalize

def pagerank(x, df=0.85, max_iter=30, bias=None):

    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)

    # check bias
    if bias is None:
        bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)
    else:
        bias = bias.reshape(-1,1)
        bias = A.shape[0] * bias / bias.sum()
        assert bias.shape[0] == A.shape[0]
        bias = (1 - df) * bias

    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R


In [2]:
# sentence.py
from collections import Counter
import math
import numpy as np
import scipy as sp
from scipy.sparse import csr_matrix
from sklearn.metrics import pairwise_distances

def sent_graph(sents, tokenize=None, min_count=2, min_sim=0.3, similarity=None, vocab_to_idx=None, verbose=False):
    if vocab_to_idx is None:
        idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    else:
        idx_to_vocab = [vocab for vocab, _ in sorted(vocab_to_idx.items(), key=lambda x:x[1])]

    x = vectorize_sents(sents, tokenize, vocab_to_idx)
    if similarity == 'cosine':
        x = numpy_cosine_similarity_matrix(x, min_sim, verbose, batch_size=1000)
    else:
        x = numpy_textrank_similarity_matrix(x, min_sim, verbose, batch_size=1000)
    return x

def vectorize_sents(sents, tokenize, vocab_to_idx):
    rows, cols, data = [], [], []
    for i, sent in enumerate(sents):
        counter = Counter(tokenize(sent))
        for token, count in counter.items():
            j = vocab_to_idx.get(token, -1)
            if j == -1:
                continue
            rows.append(i)
            cols.append(j)
            data.append(count)
    n_rows = len(sents)
    n_cols = len(vocab_to_idx)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

def numpy_cosine_similarity_matrix(x, min_sim=0.3, verbose=True, batch_size=1000):
    n_rows = x.shape[0]
    mat = []
    for bidx in range(math.ceil(n_rows / batch_size)):
        b = int(bidx * batch_size)
        e = min(n_rows, int((bidx+1) * batch_size))
        psim = 1 - pairwise_distances(x[b:e], x, metric='cosine')
        rows, cols = np.where(psim >= min_sim)
        data = psim[rows, cols]
        mat.append(csr_matrix((data, (rows, cols)), shape=(e-b, n_rows)))
        if verbose:
            print('\rcalculating cosine sentence similarity {} / {}'.format(b, n_rows), end='')
    mat = sp.sparse.vstack(mat)
    if verbose:
        print('\rcalculating cosine sentence similarity was done with {} sents'.format(n_rows))
    return mat

def numpy_textrank_similarity_matrix(x, min_sim=0.3, verbose=True, min_length=1, batch_size=1000):
    n_rows, n_cols = x.shape

    # Boolean matrix
    rows, cols = x.nonzero()
    data = np.ones(rows.shape[0])
    z = csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

    # Inverse sentence length
    size = np.asarray(x.sum(axis=1)).reshape(-1)
    size[np.where(size <= min_length)] = 10000
    size = np.log(size)

    mat = []
    for bidx in range(math.ceil(n_rows / batch_size)):

        # slicing
        b = int(bidx * batch_size)
        e = min(n_rows, int((bidx+1) * batch_size))

        # dot product
        inner = z[b:e,:] * z.transpose()

        # sentence len[i,j] = size[i] + size[j]
        norm = size[b:e].reshape(-1,1) + size.reshape(1,-1)
        norm = norm ** (-1)
        norm[np.where(norm == np.inf)] = 0

        # normalize
        sim = inner.multiply(norm).tocsr()
        rows, cols = (sim >= min_sim).nonzero()
        data = np.asarray(sim[rows, cols]).reshape(-1)

        # append
        mat.append(csr_matrix((data, (rows, cols)), shape=(e-b, n_rows)))

        if verbose:
            print('\rcalculating textrank sentence similarity {} / {}'.format(b, n_rows), end='')

    mat = sp.sparse.vstack(mat)
    if verbose:
        print('\rcalculating textrank sentence similarity was done with {} sents'.format(n_rows))

    return mat

def graph_with_python_sim(tokens, verbose, similarity, min_sim):
    if similarity == 'cosine':
        similarity = cosine_sent_sim
    elif callable(similarity):
        similarity = similarity
    else:
        similarity = textrank_sent_sim

    rows, cols, data = [], [], []
    n_sents = len(tokens)
    for i, tokens_i in enumerate(tokens):
        if verbose and i % 1000 == 0:
            print('\rconstructing sentence graph {} / {} ...'.format(i, n_sents), end='')
        for j, tokens_j in enumerate(tokens):
            if i >= j:
                continue
            sim = similarity(tokens_i, tokens_j)
            if sim < min_sim:
                continue
            rows.append(i)
            cols.append(j)
            data.append(sim)
    if verbose:
        print('\rconstructing sentence graph was constructed from {} sents'.format(n_sents))
    return csr_matrix((data, (rows, cols)), shape=(n_sents, n_sents))

def textrank_sent_sim(s1, s2):
    n1 = len(s1)
    n2 = len(s2)
    if (n1 <= 1) or (n2 <= 1):
        return 0
    common = len(set(s1).intersection(set(s2)))
    base = math.log(n1) + math.log(n2)
    return common / base

def cosine_sent_sim(s1, s2):
    if (not s1) or (not s2):
        return 0

    s1 = Counter(s1)
    s2 = Counter(s2)
    norm1 = math.sqrt(sum(v ** 2 for v in s1.values()))
    norm2 = math.sqrt(sum(v ** 2 for v in s2.values()))
    prod = 0
    for k, v in s1.items():
        prod += v * s2.get(k, 0)
    return prod / (norm1 * norm2)


In [3]:
# summarizer.py
import numpy as np

class KeywordSummarizer:
    def __init__(self, sents=None, tokenize=None, min_count=2,
        window=-1, min_cooccurrence=2, vocab_to_idx=None,
        df=0.85, max_iter=30, verbose=False):

        self.tokenize = tokenize
        self.min_count = min_count
        self.window = window
        self.min_cooccurrence = min_cooccurrence
        self.vocab_to_idx = vocab_to_idx
        self.df = df
        self.max_iter = max_iter
        self.verbose = verbose

        if sents is not None:
            self.train_textrank(sents)

    def train_textrank(self, sents, bias=None):

        g, self.idx_to_vocab = word_graph(sents,
            self.tokenize, self.min_count,self.window,
            self.min_cooccurrence, self.vocab_to_idx, self.verbose)
        self.R = pagerank(g, self.df, self.max_iter, bias).reshape(-1)
        if self.verbose:
            print('trained TextRank. n words = {}'.format(self.R.shape[0]))

    def keywords(self, topk=30):

        if not hasattr(self, 'R'):
            raise RuntimeError('Train textrank first or use summarize function')
        idxs = self.R.argsort()[-topk:]
        keywords = [(self.idx_to_vocab[idx], self.R[idx]) for idx in reversed(idxs)]
        return keywords

    def summarize(self, sents, topk=30):

        self.train_textrank(sents)
        return self.keywords(topk)


class KeysentenceSummarizer:

    def __init__(self, sents=None, tokenize=None, min_count=2,
        min_sim=0.3, similarity=None, vocab_to_idx=None,
        df=0.85, max_iter=30, verbose=False):

        self.tokenize = tokenize
        self.min_count = min_count
        self.min_sim = min_sim
        self.similarity = similarity
        self.vocab_to_idx = vocab_to_idx
        self.df = df
        self.max_iter = max_iter
        self.verbose = verbose

        if sents is not None:
            self.train_textrank(sents)

    def train_textrank(self, sents, bias=None):

        g = sent_graph(sents, self.tokenize, self.min_count,
            self.min_sim, self.similarity, self.vocab_to_idx, self.verbose)
        self.R = pagerank(g, self.df, self.max_iter, bias).reshape(-1)
        if self.verbose:
            print('trained TextRank. n sentences = {}'.format(self.R.shape[0]))

    def summarize(self, sents, topk=30, bias=None):

        n_sents = len(sents)
        if isinstance(bias, np.ndarray):
            if bias.shape != (n_sents,):
                raise ValueError('The shape of bias must be (n_sents,) but {}'.format(bias.shape))
        elif bias is not None:
            raise ValueError('The type of bias must be None or numpy.ndarray but the type is {}'.format(type(bias)))

        self.train_textrank(sents, bias)
        idxs = self.R.argsort()[-topk:]
        keysents = [(idx, self.R[idx], sents[idx]) for idx in reversed(idxs)]
        return keysents


In [4]:
# utils.py
from collections import Counter
from scipy.sparse import csr_matrix
import numpy as np


def scan_vocabulary(sents, tokenize=None, min_count=2):

    counter = Counter(w for sent in sents for w in tokenize(sent))
    counter = {w:c for w,c in counter.items() if c >= min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

def tokenize_sents(sents, tokenize):

    return [tokenize(sent) for sent in sents]

def vectorize(tokens, vocab_to_idx):

    rows, cols, data = [], [], []
    for i, tokens_i in enumerate(tokens):
        for t, c in Counter(tokens_i).items():
            j = vocab_to_idx.get(t, -1)
            if j == -1:
                continue
            rows.append(i)
            cols.append(j)
            data.append(c)
    n_sents = len(tokens)
    n_terms = len(vocab_to_idx)
    x = csr_matrix((data, (rows, cols)), shape=(n_sents, n_terms))
    return x


In [5]:
# word.py
from collections import defaultdict
from scipy.sparse import csr_matrix

def word_graph(sents, tokenize=None, min_count=2, window=2, min_cooccurrence=2, vocab_to_idx=None, verbose=False):

    if vocab_to_idx is None:
        idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    else:
        idx_to_vocab = [vocab for vocab, _ in sorted(vocab_to_idx.items(), key=lambda x:x[1])]

    tokens = tokenize_sents(sents, tokenize)
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence, verbose)
    return g, idx_to_vocab

def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2, verbose=False):

    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        if verbose and s % 1000 == 0:
            print('\rword cooccurrence counting {}'.format(s), end='')
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    if verbose:
        print('\rword cooccurrence counting from {} sents was done'.format(s+1))
    return dict_to_mat(counter, n_vocabs, n_vocabs)

def dict_to_mat(d, n_rows, n_cols):

    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))


In [14]:
with open('./lalaland_komoran.txt', encoding='utf-8') as f:
    sents = [sent.strip() for sent in f]

with open('./lalaland.txt', encoding='utf-8') as f:
    texts = [sent.strip() for sent in f]

print(len(sents), len(texts))

15595 15595


In [15]:
def komoran_tokenize(sent):
    words = sent.split()
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

keyword_extractor = KeywordSummarizer(
    tokenize = komoran_tokenize,
    window = -1,
    verbose = False
)
keywords = keyword_extractor.summarize(sents, topk=30)
for word, rank in keywords:
    print('{} ({:.3})'.format(word, rank))

영화/NNG (1.73e+02)
보/VV (1.29e+02)
좋/VA (65.5)
하/VV (52.0)
것/NNB (47.4)
같/VA (45.4)
영화/NNP (43.8)
음악/NNG (43.6)
꿈/NNG (41.4)
있/VV (40.8)
없/VA (35.9)
마지막/NNG (31.9)
수/NNB (30.1)
사랑/NNG (28.3)
아름답/VA (26.5)
현실/NNG (24.8)
되/VV (23.9)
노래/NNG (23.4)
생각/NNG (23.2)
스토리/NNP (21.4)
번/NNB (20.3)
거/NNB (19.7)
최고/NNG (19.2)
때/NNG (19.1)
사람/NNG (19.0)
여운/NNP (17.5)
뮤지컬/NNP (16.9)
나오/VV (16.5)
듯/NNB (16.1)
영상미/NNG (16.0)


In [16]:
summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenize,
    min_sim = 0.5,
    verbose = True
)
keysents = summarizer.summarize(sents)
for idx, rank, komoran_sent in keysents:
    print('#{} ({:.3}) : {}'.format(idx, rank, texts[idx]), end='\n\n')

calculating textrank sentence similarity was done with 15595 sents
trained TextRank. n sentences = 15595
#5861 (6.12) : 사랑에 대해 다시 한 번 생각해 볼 수 있게 하는 영화인 것 같습니다 장면 처리도 좋았어요 여운이 많이 남는 영화입니다 꼭 보세요

#5947 (5.8) : 아 진짜 평점 처음 써본다 진짜 후회 안할 영화 나중에 다시 봐도 좋을것 같다 오프닝에서 신나는 노래부터 마지막의 상상 하는 씬까지 너무 좋음 결말이 여운이 있다고 해야하나 슬프다고 해야하나 꿈얘기 할때 현실성 있어서 눈물날뻔 결말이짱

#5076 (5.69) : 옛날 영화같은 느낌의 기법 사람을 행복하게 만드는 음악 약간의 촌스러움이 마음을 간지를 수 있는 지극히 현실적인 사랑과 꿈이야기를 로맨틱하게 풀어낸 영화로 연말 영화로 보기 좋은 것 같아요

#6665 (5.41) : 인생영화다 노래도 너무 좋고 배우 소품 배경 장면들 하나하나 맘에 안 드는게 없다 ㅠㅠ 특히 마지막 셉oo에서의 내용은 진짜 잊을 수가 없을 거같다 보고나면 먹먹하고 안타까운 느낌이 드는데 그래도 황홀하고 아름다운 영화다

#9271 (5.28) : 연출 음악 영상미 엔딩은 정말 좋았다 마지막에 남녀주인공이 나눈 눈빛이 아직도 잊혀지지않을만큼 여운이 남는 영화였다 초중반 약간 지루하긴했었다 배우들 춤연습을 많이한게보였음 꿈 성공 과 사랑을 다 가질수 없다는것을 현실적으로보여준 영화가아니었나싶다

#13909 (5.12) : 인생 최고의 영화 또보고싶다 영상미 음악 스토리 다 좋아요

#5922 (4.99) : 정말 영상미랑 음악은 최고였다 그리고 신선했다 음악이 너무 멋있어서 연기를 봐야 할지 노래를 들어야 할지 모를 정도로 그리고 보고 나서 생각 좀 많아진 영화 정말 이 연말에 보기 좋은 영화 인 것 같다

#11408 (4.88) : 진짜 그냥 좋았던 영화 두번봐도 재밌을영화

#12362 (4.88) : 보고 난 후

In [6]:
import pandas as pd
f=pd.read_csv('/content/final_train_data_3.txt', delimiter = '\t')
f.head()

,review_id,review,rating,label
0,2731670,특별하진 않지만 사랑스러운 작품,7,2
1,4709715,실수를 연발하는그녀가 이제 더이상 사랑스럽다기보다 ㅇ나ㅆ,2,1
2,111222,무드만 있고 현상이없다. 스타일만있고 공허하다.,5,2
3,132464,충격,5,2
4,64730,가볍고 유쾌한 영화.,8,2


In [7]:
texts = f['review']
texts[1]

'실수를 연발하는그녀가 이제 더이상 사랑스럽다기보다 ㅇ나ㅆ'

In [9]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
     |████████████████████████████████| 19.4MB 40.8MB/s 
     |████████████████████████████████| 3.8MB 41.2MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ bash -x
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb

In [10]:
len(texts)

119999

In [63]:
from konlpy.tag import Mecab
m = Mecab()
f1 = open('/content/review_mecab.txt', 'w')

for i in range(len(texts)):
    sent = ''
    tmp = m.pos(texts[i])
    for j in range(len(tmp)):
        sent += tmp[j][0] + '/' + tmp[j][1] + ' '
    f1.write(sent)
    f1.write("\n")

f1.close()

In [64]:
with open('/content/review_mecab.txt', encoding='utf-8') as f:
    sents = [sent.strip() for sent in f]

In [65]:
sents[0]

'특별/NNG 하/XSA 진/EC+JX 않/VX 지만/EC 사랑/NNG 스러운/XSA+ETM 작품/NNG'

In [66]:
f2 = open("/content/review.txt", 'w')

for i in range(len(texts)):
    f2.write(str(texts[i]))
    f2.write("\n")

f2.close()

In [67]:
with open('/content/review.txt', encoding='utf-8') as f:
    texts = [sent.strip() for sent in f]

In [68]:
texts[0]

'특별하진 않지만 사랑스러운 작품'

komoran 품사태깅 시 리뷰가 nan인경우 예외처리 추가해야함.

In [ ]:
from PyKomoran import *
pko = Komoran("EXP")
sents = ""
for i in range(116350, len(texts)):
    sents[i] = pko.get_plain_text(texts[i])

In [71]:
f1 = open("/content/review_komoran.txt", 'w')
for i in range(len(sents)):
    f1.write(sents[i])
f1.close()

f2 = open("/content/review.txt", 'w')
for i in range(len(texts)):
    f2.write(str(texts[i]))
    f2.write("\n")
f2.close()

In [70]:
with open('/content/review_mecab.txt', encoding='utf-8') as f:
    sents = [sent.strip() for sent in f.readlines()]

with open('/content/review.txt', encoding='utf-8') as f:
    texts = [sent.strip() for sent in f]

print(len(sents), len(texts))

119999 119999


In [71]:
def mecab_tokenizer(sent):
    words = sent.split()
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

keyword_extractor = KeywordSummarizer(
    tokenize = mecab_tokenizer,
    window = -1,
    verbose = False
)
keywords = keyword_extractor.summarize(sents, topk=30)
for word, rank in keywords:
    print('{} ({:.3})'.format(word, rank))

영화/NNG (9.21e+02)
없/VA (2.63e+02)
보/VV (2.59e+02)
하/VV (1.86e+02)
것/NNB (1.83e+02)
좋/VA (1.61e+02)
수/NNB (1.47e+02)
같/VA (1.42e+02)
있/VV (1.4e+02)
년/NNBC (1.21e+02)
때/NNG (1.1e+02)
거/NNB (1.03e+02)
연기/NNG (92.3)
스토리/NNG (92.1)
듯/NNB (85.4)
되/VV (83.4)
감독/NNG (79.2)
작품/NNG (79.1)
점/NNBC (78.6)
있/VA (78.5)
생각/NNG (73.9)
게/NNB+JKS (72.9)
지루/XR (72.4)
장면/NNG (70.8)
사람/NNG (70.6)
배우/NNG (69.9)
말/NNG (69.6)
액션/NNG (69.0)
봤/VV+EP (67.1)
내용/NNG (65.0)


In [72]:
keyword_extractor = KeywordSummarizer(
    tokenize=lambda x:x.split(),
    verbose = False
)
keywords = keyword_extractor.summarize(sents, topk=30)
for word, rank in keywords:
    print('{} ({:.3})'.format(word, rank))

./SF (1.22e+03)
영화/NNG (4.78e+02)
의/JKG (4.38e+02)
는/ETM (4.06e+02)
이/JKS (3.97e+02)
고/EC (3.74e+02)
에/JKB (3.19e+02)
다/EF (3.12e+02)
,/SC (2.96e+02)
도/JX (2.8e+02)
./SY (2.75e+02)
가/JKS (2.73e+02)
을/JKO (2.61e+02)
는/JX (2.5e+02)
은/JX (2.15e+02)
한/XSA+ETM (2.15e+02)
../SY (1.95e+02)
이/VCP (1.93e+02)
를/JKO (1.88e+02)
들/XSN (1.65e+02)
하/XSV (1.62e+02)
보/VV (1.41e+02)
하/XSA (1.39e+02)
게/EC (1.36e+02)
없/VA (1.36e+02)
은/ETM (1.22e+02)
적/XSN (1.2e+02)
지/EC (1.15e+02)
다/EC (1.14e+02)
었/EP (1.13e+02)


In [73]:
summarizer = KeysentenceSummarizer(
    tokenize = mecab_tokenizer,
    min_sim = 0.5,
    verbose = True
)
keysents = summarizer.summarize(sents)
for idx, rank, komoran_sent in keysents:
    print('#{} ({:.3}) : {}'.format(idx, rank, texts[idx]), end='\n\n')

calculating textrank sentence similarity was done with 119999 sents
trained TextRank. n sentences = 119999
#77186 (10.1) : 참으로 좋은 영화라 하지 않을 수 없는 것은 아니지 않습니다.

#30846 (9.08) : 다시 보고 싶은 영화

#35696 (9.08) : 지금 보니 프리메이슨 영화네

#14363 (9.08) : 다시 보고싶은 영화..

#114666 (9.08) : 또 보고 싶은 영화!

#5124 (9.08) : 다시 한번더 보고싶은 영화

#112871 (9.08) : 다시 한번 더 보고픈 영화

#10586 (9.08) : 영화 보는 내내 빙그레~ㅋㅋㅋ

#117941 (9.08) : 보다가 나가는 영화??

#41621 (9.08) : 꼭 다시보고픈영화..

#69219 (9.08) : 생각없이보는영화

#65802 (9.08) : 이런 영화는 다시 보고 싶지 않다.

#52407 (9.08) : 이 영화를 왜 보았을까

#95608 (9.08) : 영화만 보자.... -,-;;;

#18708 (9.08) : 다시보고싶은 영화

#3391 (9.08) : 내가 이 영화를 왜 보았나?

#108615 (9.08) : 다시 보고 싶은 영화!!

#12106 (9.08) : 다시 보고 싶은 영화

#84911 (9.08) : 다시보고싶은영화,,

#81460 (8.92) : 중간중간뻔하고지루한장면도잇엇다하지만 연애의 정석을 보여주는것같다 2편째부터 영화에빠져들더니 다보고나니까 기분이좋다 역시 남자랑여자는 친구가 될수없는건가사랑하는 내사람이 갑자기 막보고싶어진다 역시대작은 대작이구나 지금봐도 명작인걸보면

#90357 (8.3) : 이영화 볼수있는데 없을까요??다시한번 꼭 보고싶은데...

#50477 (8.3) : 영화인가? 별 없음

#9163 (8.3) : 아무것도 없는 영화...

#113394 (8.3) : 느낌없는 영화

#84559 (8